In [ ]:
import sys
import numpy as np
import pandas as pd
from matplotlib.pyplot import imread
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time
import random

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K

from PIL import Image
import numpy.random as rng

In [ ]:
train_folder = 'Assets/champions/'
val_folder = 'Assets/example/'
save_path = 'data/'

In [ ]:
def loadimgs(path):
    '''
    path => Path of train directory(champions)
    '''
    images= []
    labels= []
    
    for filename in os.listdir(path):
        image_path = os.path.join(path, filename)
        label=filename
        image = imread(image_path)
        
        #resize images
        dim = (44,44)
        image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
        
        images.append(image)
        label = label.replace('.png','')
        labels.append(label)
        
    return images,labels

In [ ]:
images, labels = loadimgs(train_folder)

together={}
for t in range(148):
    together[labels[t]] = images[t]

images = np.array(images)
labels = np.array(labels)
                  
images2 = images.copy()
images2 = images2[::-1]
labels2 = labels.copy()
labels2 = labels2[::-1]

def get_batch(y):
    
    pairs = [np.zeros((148, 44, 44,3)) for i in range(2)]
    targets = np.zeros((148,))

    input1 = [images[y]]*148
    input2 = images.copy()
    targets[y]=1
        
    pairs[0] = input1
    pairs[1] = input2
    return pairs, targets

In [ ]:
im = Image.open(r"Assets/example/screenshot.png") 
leftgroup=[]
rightgroup=[]

for i in range(5):
    left = 920
    right = 964
    top = 860+i*44
    bottom = 860+(i+1)*44
    im1 = im.crop((left, top, right, bottom)) 
    # Shows the image in image viewer 
    #im1.show() 
    im1 = np.array(im1)
    temp = [im1]*148
    np.asarray(temp)
    leftgroup.append(temp)
leftgroup = np.asarray(leftgroup)    

for i in range(5):
    left2 = 970
    right2 = 1014
    top2 = 860+i*44
    bottom2 = 860+(i+1)*44
    im2 = im.crop((left2, top2, right2, bottom2)) 
    # Shows the image in image viewer 
    #im2.show() 
    im2 = np.array(im2)
    temp2 = [im2]*148
    np.asarray(temp2)
    rightgroup.append(temp2)
rightgroup = np.asarray(rightgroup)    

In [ ]:
def initialize_weights(shape, name=None):
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [ ]:
def initialize_bias(shape, name=None):
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [ ]:
def get_siamese_model(input_shape):
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    
    model.add(Conv2D(64, (4,4), activation='relu', #128
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
#    model.add(Conv2D(128, (2,2), activation='relu', kernel_initializer=initialize_weights,
#                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
#    model.add(MaxPooling2D())
    
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights, #256
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))

    
    
    # Generate the feature vectors for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the feature vectors
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [ ]:
model = get_siamese_model((44,44,3))
model.summary()

In [ ]:
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

### Loading the train tensors

In [ ]:
def make_oneshot_task(N,c,m):
    if (c=='right'):
        test_image = rightgroup[m]
        print('\n Original Image: ')
        plt.imshow(rightgroup[m][0])
        plt.show()
        
    else:
        test_image = leftgroup[m]
        print('\n Original Image: ')
        plt.imshow(leftgroup[m][0])
        plt.show()
        
    support_set = images
    targets = np.zeros((N,))
    pairs = [test_image,support_set]

    return pairs, targets

In [ ]:
def test_oneshot(model,N,c,m):
    inputs, targets = make_oneshot_task(N,c,m)
    probs = model.predict(inputs)
    return probs

In [ ]:
n_iter = 200 # Number of iterations

In [ ]:
model_path = './weights/'

In [ ]:
print("Starting training process!")
print("-------------------------------------")
t_start = time.time()
for i in range(1, n_iter+1):
    for t in range(148):
        (inputs,targets) = get_batch(t)
        loss = model.train_on_batch(inputs, targets)
        print("Train Loss: {0}".format(loss)) 
    model.save_weights(os.path.join(model_path, 'weights.{}.h5'.format(i)))

In [ ]:
model.load_weights(os.path.join('./weights/', 'weights.10.h5'))

In [ ]:
for k in range(5):
    print("\n{}. Champion of the Right Group\n".format(k+1))
    probs= test_oneshot(model,148,'right',k)
    total={}
    for t in range(148):
        total[labels[t]] = probs[t]
    final = sorted(total.items(), key=lambda x: x[1], reverse=True)
    key=str(final[0][0])
    print('\n Predicted Image: ')
    plt.imshow(together[key])
    plt.show()
    
for k in range(5):
    print("\n{}. Champion of the Left Group\n".format(k+1))
    probs= test_oneshot(model,148,'left',k)
    total={}
    for t in range(148):
        total[labels[t]] = probs[t]
    final = sorted(total.items(), key=lambda x: x[1], reverse=True)
    print(final[0])
    key=str(final[0][0])
    print('\n Predicted Image: ')
    plt.imshow(together[key])
    plt.show()